In [1]:
import os
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import glob 
import pytz
import dask.array as da

In [2]:
# Set the folder path where your .nc files are located
folder_path = '/home/paulc600/local/Easymore/HYPE_merged_easymore/*.nc'

In [3]:
# Set the output folder path where you want to save the text files
output_folder_path = '/home/paulc600/scratch/HYPE/Landsat/06_maxmin/'

In [4]:
# Get a list of all .nc files in the folder
file_list = glob.glob(folder_path)

In [5]:
# Open the .nc files using xarray's open_mfdataset
dataset = xr.open_mfdataset(file_list, combine='by_coords')


In [6]:
dataset

<xarray.Dataset>
Dimensions:        (time: 341496, ID: 473)
Coordinates:
  * time           (time) datetime64[ns] 1980-01-01T13:00:00 ... 2018-12-31T1...
  * ID             (ID) float64 1.136e+05 1.136e+05 ... 1.144e+05 1.144e+05
Data variables:
    latitude       (time, ID) float64 dask.array<chunksize=(8760, 473), meta=np.ndarray>
    longitude      (time, ID) float64 dask.array<chunksize=(8760, 473), meta=np.ndarray>
    precipitation  (time, ID) float32 dask.array<chunksize=(8760, 473), meta=np.ndarray>
    temperature    (time, ID) float32 dask.array<chunksize=(8760, 473), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.0.5 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    Author:       The data were written by author name
    History:      Created Thu Jun 22 10:58:17 2023
    history:      Tue Jun 27 00:26:07 2023: cdo mergetime SMMFullTime_easymor...
    CDO:          Climate Data Operators version 2.0.5 (https://mpimet.mpg.de...

In [7]:
# Extract the time coordinate from the dataset
time_coords = dataset.coords['time']

In [8]:
# Convert the object array to pandas Timestamp objects
time_values = pd.to_datetime(time_coords.data)

In [9]:
# Define the UTC and MST time zones
utc_tz = pytz.UTC
mst_tz = pytz.timezone('US/Mountain')

In [10]:
# Convert time from UTC to MST using pandas Timestamp
converted_times = [utc_tz.localize(dt).astimezone(mst_tz) for dt in time_values]

In [11]:
# Create a new dataset with the converted time coordinate
ds_converted = dataset.assign_coords(time=converted_times)

In [12]:
ds_converted

<xarray.Dataset>
Dimensions:        (time: 341496, ID: 473)
Coordinates:
  * time           (time) object 1980-01-01T06:00:00-07:00 ... 2018-12-31T05:...
  * ID             (ID) float64 1.136e+05 1.136e+05 ... 1.144e+05 1.144e+05
Data variables:
    latitude       (time, ID) float64 dask.array<chunksize=(8760, 473), meta=np.ndarray>
    longitude      (time, ID) float64 dask.array<chunksize=(8760, 473), meta=np.ndarray>
    precipitation  (time, ID) float32 dask.array<chunksize=(8760, 473), meta=np.ndarray>
    temperature    (time, ID) float32 dask.array<chunksize=(8760, 473), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.0.5 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    Author:       The data were written by author name
    History:      Created Thu Jun 22 10:58:17 2023
    history:      Tue Jun 27 00:26:07 2023: cdo mergetime SMMFullTime_easymor...
    CDO:          Climate Data Operators version 2.0.5 (https://mpimet.mpg.de...

In [13]:
# Resample the temperature array to daily maximum and minimum
temperature_daily_max = ds_converted['temperature'].resample(time='D').max(dim='time')
temperature_daily_min = ds_converted['temperature'].resample(time='D').min(dim='time')

In [14]:
temperature_daily_max

<xarray.DataArray 'temperature' (time: 14245, ID: 473)>
dask.array<stack, shape=(14245, 473), dtype=float32, chunksize=(1, 473), chunktype=numpy.ndarray>
Coordinates:
  * ID       (ID) float64 1.136e+05 1.136e+05 1.136e+05 ... 1.144e+05 1.144e+05
  * time     (time) object 315558000000000000 ... 1546239600000000000
Attributes:
    long_name:  Forecast: Air temperature
    units:      deg_C

In [15]:
# Convert the resampled data variables to pandas DataFrames
temperature_daily_max_df = temperature_daily_max.to_dataframe().unstack('ID')['temperature']
temperature_daily_min_df = temperature_daily_min.to_dataframe().unstack('ID')['temperature']

In [16]:
# read shapefile with extra river ID given to catchments without rivers
New_ID=gpd.read_file('/home/paulc600/local/SMM/Modified_HYPE/Modified_SMMcat.shp')

In [17]:
# Convert shapefile seg_Nhm to integer
New_ID['seg_nhm'] = New_ID['seg_nhm'].astype(int)

In [18]:
# Reset the index to remove time and column names from the index
temperature_daily_max_df = temperature_daily_max_df.reset_index()
temperature_daily_max_df.loc[-1] = temperature_daily_max_df.columns
temperature_daily_max_df.sort_index(inplace=True)

temperature_daily_min_df = temperature_daily_min_df.reset_index()
temperature_daily_min_df.loc[-1] = temperature_daily_min_df.columns
temperature_daily_min_df.sort_index(inplace=True)

In [20]:
# Convert the first row (excluding the first column) to integers and truncate the decimals
temperature_daily_max_df.iloc[0, 1:] = temperature_daily_max_df.iloc[0, 1:].astype(int).astype(str).str.split('.').str[0]
temperature_daily_min_df.iloc[0, 1:] = temperature_daily_min_df.iloc[0, 1:].astype(int).astype(str).str.split('.').str[0]

In [25]:
temperature_daily_max_df

ID,time,113602.0,113598.0,113588.0,113584.0,113577.0,113579.0,113585.0,113553.0,113552.0,...,114230.0,114240.0,114258.0,114294.0,114299.0,114309.0,114313.0,114322.0,114388.0,114417.0
-1,time,113602,113598,113588,113584,113577,113579,113585,113553,113552,...,114230,114240,114258,114294,114299,114309,114313,114322,114388,114417
0,1980-01-01 00:00:00-07:00,2.098747,2.21415,2.220917,2.227792,2.293574,1.877796,2.079603,1.52113,1.227171,...,-0.012041,-2.161228,0.186475,-1.419923,-1.806059,-0.017913,0.152926,-0.190182,1.486813,-0.423516
1,1980-01-02 00:00:00-07:00,3.548144,3.860312,3.936722,3.761164,3.805379,3.395295,3.469882,2.716048,2.439829,...,-3.829781,-5.235143,-3.646789,-4.648786,-4.826409,-3.53615,-3.366373,-3.728445,-0.782822,-4.671653
2,1980-01-03 00:00:00-07:00,0.345704,0.235007,0.188766,0.450745,0.848725,0.856503,0.385021,1.029853,0.919128,...,-0.020518,0.605797,-0.277506,-0.457247,-0.372206,-0.539213,-0.625303,-1.320147,0.872329,0.549265
3,1980-01-04 00:00:00-07:00,-3.007683,-2.918182,-2.950607,-2.590651,-1.832423,-2.410624,-2.591058,-2.738928,-3.058579,...,-7.511217,-7.634901,-7.006943,-8.352027,-8.414834,-7.390435,-6.849273,-7.05183,-4.624328,-8.160263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14240,2018-12-27 00:00:00-07:00,-4.16795,-4.130198,-4.113281,-4.179012,-4.234842,-4.353816,-5.047713,-4.899006,-5.73369,...,-3.84989,-5.599911,-3.816716,-4.278603,-4.43516,-4.035249,-3.971967,-4.36756,-3.679245,-4.982377
14241,2018-12-28 00:00:00-07:00,-1.53467,-1.619724,-1.629028,-1.537171,-1.353207,-1.686703,-2.762412,-2.891262,-4.071136,...,-2.407936,-3.312369,-2.465053,-2.630273,-2.500078,-2.673093,-2.897048,-3.572976,-2.071374,-2.502706
14242,2018-12-29 00:00:00-07:00,6.024182,6.095463,6.111816,6.110557,6.268211,5.633241,3.944397,3.809276,2.310412,...,2.650857,1.70013,2.775895,1.724859,1.697376,2.159538,2.50302,1.924035,5.166318,2.403457
14243,2018-12-30 00:00:00-07:00,-0.286776,0.132351,0.362991,-0.294536,-0.297359,-0.132394,0.30771,0.190542,0.242328,...,1.119921,1.192724,0.685654,0.850972,0.953649,0.620445,0.469993,0.521335,3.529174,1.669567


In [22]:
New_ID

,hru_nhm,seg_nhm,POI_ID,hru_id,hru_segmen,hru_id_tb,hru_segm_1,Type_NCA,HUC04,Coastal_HR,Shape_Leng,Shape_Area,geometry
0,113602,58183,6.500040e+13,113,3,3651,1723,0,0904,0,28637.472967,3.652090e+07,"POLYGON ((-112.87990 49.58915, -112.87988 49.5..."
1,113598,58184,6.500040e+13,109,4,3647,1724,0,0904,0,22917.257393,1.750700e+07,"POLYGON ((-112.84816 49.57293, -112.84776 49.5..."
2,113588,58185,6.500040e+13,99,5,3637,1725,0,0904,0,10941.686207,5.667200e+06,"POLYGON ((-112.83280 49.56569, -112.83283 49.5..."
3,113584,58186,6.500040e+13,95,6,3633,1726,0,0904,0,47581.126463,3.803350e+07,"POLYGON ((-112.83256 49.56479, -112.83243 49.5..."
4,113577,58188,6.500040e+13,88,8,3626,1728,0,0904,0,11865.104926,2.801800e+06,"POLYGON ((-112.93812 49.51740, -112.93809 49.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,114309,58671,2.300280e+13,699,0,4358,0,1,1005,0,202460.006602,3.860121e+08,"MULTIPOLYGON (((-108.52850 49.15148, -108.5286..."
469,114313,58672,2.300280e+13,703,0,4362,0,1,1005,0,295200.002203,9.586554e+08,"MULTIPOLYGON (((-108.67155 49.02478, -108.6714..."
470,114322,58673,2.300280e+13,712,0,4371,0,1,1005,0,188640.001102,2.876408e+08,"MULTIPOLYGON (((-108.96500 49.40234, -108.9650..."
471,114388,58674,2.300280e+13,778,0,4437,0,1,1005,0,105620.001701,2.727878e+08,"POLYGON ((-110.30901 48.72789, -110.30899 48.7..."


In [26]:
# Extract the 'seg_nhm' values corresponding to 'hru_nhm' from New_ID
seg_nhm_values = New_ID.set_index('hru_nhm').loc[temperature_daily_max_df.columns[1:], 'seg_nhm'].values

In [27]:
seg_nhm_values

array([58183, 58184, 58185, 58186, 58188, 58189, 58192, 58193, 58194,
       58197, 58201, 58202, 58203, 58205, 58206, 58207, 58208, 58210,
       58212, 58213, 58214, 58216, 58217, 58218, 58219, 58221, 58222,
       58223, 58225, 58227, 58228, 58229, 58231, 58232, 58233, 58234,
       58236, 58237, 58240, 58241, 58242, 58243, 58244, 58245, 58246,
       58247, 58248, 58249, 58250, 58251, 58252, 58253, 58254, 58255,
       58256, 58257, 58258, 58259, 58260, 58261, 58262, 58263, 58264,
       58265, 58266, 58267, 58268, 58269, 58270, 58271, 58272, 58273,
       58274, 58275, 58276, 58277, 58278, 58279, 58280, 58281, 58282,
       58283, 58284, 58285, 58286, 58287, 58288, 58289, 58290, 58291,
       58292, 58293, 58294, 58295, 58296, 58297, 58298, 58299, 58300,
       58301, 58302, 58303, 58304, 58305, 58306, 58307, 58308, 58309,
       58310, 58311, 58312, 58313, 58314, 58315, 58316, 58317, 58318,
       58319, 58320, 58321, 58322, 58323, 58324, 58325, 58326, 58327,
       58328, 58329,

In [30]:
# Replace the first row in data_df with the seg_nhm_values
temperature_daily_max_df.iloc[0, 1:] = seg_nhm_values
temperature_daily_min_df.iloc[0, 1:] = seg_nhm_values

In [32]:
temperature_daily_min_df

ID,time,113602.0,113598.0,113588.0,113584.0,113577.0,113579.0,113585.0,113553.0,113552.0,...,114230.0,114240.0,114258.0,114294.0,114299.0,114309.0,114313.0,114322.0,114388.0,114417.0
-1,time,58183,58184,58185,58186,58188,58189,58192,58193,58194,...,58666,58667,58668,58669,58670,58671,58672,58673,58674,58675
0,1980-01-01 00:00:00-07:00,-11.465392,-11.218625,-11.16449,-11.456625,-11.969514,-11.777235,-10.608739,-11.408575,-11.872603,...,-12.651553,-13.277964,-12.371169,-12.071502,-12.156215,-13.272165,-13.657717,-14.664277,-13.359089,-13.79773
1,1980-01-02 00:00:00-07:00,-9.417672,-9.106274,-9.03804,-9.215179,-9.238105,-9.557716,-10.050452,-11.112912,-13.422135,...,-8.171641,-8.943862,-8.204899,-8.966001,-9.044941,-8.642238,-8.815523,-10.226023,-12.693686,-8.637051
2,1980-01-03 00:00:00-07:00,-13.278946,-13.12505,-13.134918,-13.16331,-13.432444,-14.046908,-13.597026,-14.668213,-15.461432,...,-9.843618,-7.772574,-10.214027,-9.294769,-9.055331,-10.511141,-11.012546,-11.305754,-11.397153,-9.081297
3,1980-01-04 00:00:00-07:00,-19.240791,-19.667238,-19.851074,-18.201914,-15.511579,-14.732224,-17.087019,-15.389911,-18.240383,...,-15.981263,-13.346998,-16.244488,-13.34285,-13.455397,-17.148319,-17.307774,-16.182245,-17.071726,-15.446552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14240,2018-12-27 00:00:00-07:00,-9.962531,-10.321604,-10.323013,-10.191634,-10.08293,-9.598008,-10.9234,-10.967459,-13.98097,...,-11.218206,-13.903594,-11.374362,-10.974055,-12.063248,-10.960636,-10.97948,-10.886075,-10.550346,-13.194924
14241,2018-12-28 00:00:00-07:00,-12.703018,-12.214521,-12.141495,-12.205685,-11.919958,-12.090093,-14.493673,-14.611652,-19.287045,...,-9.19869,-12.268656,-9.219467,-9.106403,-9.483406,-9.199728,-9.391081,-9.738336,-9.396876,-11.885173
14242,2018-12-29 00:00:00-07:00,-0.531342,-0.515056,-0.504623,-0.584275,-0.706625,-1.260752,-2.886656,-2.432311,-4.017458,...,-5.632041,-10.017053,-5.402991,-6.313903,-6.494508,-6.917508,-6.789839,-7.827494,-3.910577,-8.357614
14243,2018-12-30 00:00:00-07:00,-11.909644,-11.886831,-11.865402,-11.993932,-12.177103,-12.583241,-13.372872,-12.873341,-13.111849,...,-15.787368,-15.97335,-15.936116,-16.645575,-16.687077,-16.291897,-15.754715,-16.145012,-11.782701,-15.114392


In [33]:
# Move the first row into the index and the first column into the index
temperature_daily_max_df = temperature_daily_max_df.rename(columns=temperature_daily_max_df.iloc[0]).drop(temperature_daily_max_df.index[0]).reset_index(drop=True)
temperature_daily_max_df = temperature_daily_max_df.set_index('time')

temperature_daily_min_df = temperature_daily_min_df.rename(columns=temperature_daily_min_df.iloc[0]).drop(temperature_daily_min_df.index[0]).reset_index(drop=True)
temperature_daily_min_df = temperature_daily_min_df.set_index('time')

In [35]:
temperature_daily_max_df

ID,58183,58184,58185,58186,58188,58189,58192,58193,58194,58197,...,58666,58667,58668,58669,58670,58671,58672,58673,58674,58675
time,,,,,,,,,,,,,,,,,,,,,
1980-01-01 00:00:00-07:00,2.098747,2.21415,2.220917,2.227792,2.293574,1.877796,2.079603,1.52113,1.227171,1.720638,...,-0.012041,-2.161228,0.186475,-1.419923,-1.806059,-0.017913,0.152926,-0.190182,1.486813,-0.423516
1980-01-02 00:00:00-07:00,3.548144,3.860312,3.936722,3.761164,3.805379,3.395295,3.469882,2.716048,2.439829,2.490557,...,-3.829781,-5.235143,-3.646789,-4.648786,-4.826409,-3.53615,-3.366373,-3.728445,-0.782822,-4.671653
1980-01-03 00:00:00-07:00,0.345704,0.235007,0.188766,0.450745,0.848725,0.856503,0.385021,1.029853,0.919128,0.445653,...,-0.020518,0.605797,-0.277506,-0.457247,-0.372206,-0.539213,-0.625303,-1.320147,0.872329,0.549265
1980-01-04 00:00:00-07:00,-3.007683,-2.918182,-2.950607,-2.590651,-1.832423,-2.410624,-2.591058,-2.738928,-3.058579,-2.281722,...,-7.511217,-7.634901,-7.006943,-8.352027,-8.414834,-7.390435,-6.849273,-7.05183,-4.624328,-8.160263
1980-01-05 00:00:00-07:00,-13.767294,-13.834729,-13.883789,-13.531507,-11.697703,-12.288984,-12.334405,-12.153723,-11.273466,-10.013642,...,-8.326643,-7.68867,-8.491439,-8.831301,-8.818301,-8.672849,-8.573922,-8.499763,-8.351196,-7.576006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27 00:00:00-07:00,-4.16795,-4.130198,-4.113281,-4.179012,-4.234842,-4.353816,-5.047713,-4.899006,-5.73369,-6.229057,...,-3.84989,-5.599911,-3.816716,-4.278603,-4.43516,-4.035249,-3.971967,-4.36756,-3.679245,-4.982377
2018-12-28 00:00:00-07:00,-1.53467,-1.619724,-1.629028,-1.537171,-1.353207,-1.686703,-2.762412,-2.891262,-4.071136,-4.007222,...,-2.407936,-3.312369,-2.465053,-2.630273,-2.500078,-2.673093,-2.897048,-3.572976,-2.071374,-2.502706
2018-12-29 00:00:00-07:00,6.024182,6.095463,6.111816,6.110557,6.268211,5.633241,3.944397,3.809276,2.310412,2.158788,...,2.650857,1.70013,2.775895,1.724859,1.697376,2.159538,2.50302,1.924035,5.166318,2.403457


In [36]:
# Convert the datetime index to a string representation with 'YYYY-MM-DD' format
temperature_daily_max_df.index = temperature_daily_max_df.index.strftime('%Y-%m-%d')
temperature_daily_min_df.index = temperature_daily_min_df.index.strftime('%Y-%m-%d')

In [37]:
temperature_daily_max_df

ID,58183,58184,58185,58186,58188,58189,58192,58193,58194,58197,...,58666,58667,58668,58669,58670,58671,58672,58673,58674,58675
time,,,,,,,,,,,,,,,,,,,,,
1980-01-01,2.098747,2.21415,2.220917,2.227792,2.293574,1.877796,2.079603,1.52113,1.227171,1.720638,...,-0.012041,-2.161228,0.186475,-1.419923,-1.806059,-0.017913,0.152926,-0.190182,1.486813,-0.423516
1980-01-02,3.548144,3.860312,3.936722,3.761164,3.805379,3.395295,3.469882,2.716048,2.439829,2.490557,...,-3.829781,-5.235143,-3.646789,-4.648786,-4.826409,-3.53615,-3.366373,-3.728445,-0.782822,-4.671653
1980-01-03,0.345704,0.235007,0.188766,0.450745,0.848725,0.856503,0.385021,1.029853,0.919128,0.445653,...,-0.020518,0.605797,-0.277506,-0.457247,-0.372206,-0.539213,-0.625303,-1.320147,0.872329,0.549265
1980-01-04,-3.007683,-2.918182,-2.950607,-2.590651,-1.832423,-2.410624,-2.591058,-2.738928,-3.058579,-2.281722,...,-7.511217,-7.634901,-7.006943,-8.352027,-8.414834,-7.390435,-6.849273,-7.05183,-4.624328,-8.160263
1980-01-05,-13.767294,-13.834729,-13.883789,-13.531507,-11.697703,-12.288984,-12.334405,-12.153723,-11.273466,-10.013642,...,-8.326643,-7.68867,-8.491439,-8.831301,-8.818301,-8.672849,-8.573922,-8.499763,-8.351196,-7.576006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,-4.16795,-4.130198,-4.113281,-4.179012,-4.234842,-4.353816,-5.047713,-4.899006,-5.73369,-6.229057,...,-3.84989,-5.599911,-3.816716,-4.278603,-4.43516,-4.035249,-3.971967,-4.36756,-3.679245,-4.982377
2018-12-28,-1.53467,-1.619724,-1.629028,-1.537171,-1.353207,-1.686703,-2.762412,-2.891262,-4.071136,-4.007222,...,-2.407936,-3.312369,-2.465053,-2.630273,-2.500078,-2.673093,-2.897048,-3.572976,-2.071374,-2.502706
2018-12-29,6.024182,6.095463,6.111816,6.110557,6.268211,5.633241,3.944397,3.809276,2.310412,2.158788,...,2.650857,1.70013,2.775895,1.724859,1.697376,2.159538,2.50302,1.924035,5.166318,2.403457


Convert to mm seperately

In [32]:
temperature_daily_min_df

seg_nhm,58219,58241,58237,58232,58214,58233,58213,58223,58221,58216,...,58448,58380,58342,58582,58405,58610,58389,58570,58579,58465
time,,,,,,,,,,,,,,,,,,,,,
1980-01-01,0.000754,0.000938,0.000873,0.000344,0.000113,0.000604,5.94e-05,0.000345,5.55e-05,0.000201,...,0.000131,0.000122,0.00011,0.000109,0,0,4.67e-09,2.65e-06,3.24e-06,6.27e-07
1980-01-02,0.000651,0.000967,0.000936,0.000403,0.000193,0.00103,0.000141,0.000282,0.000102,0.000169,...,0.000798,0.000894,0.000836,0.000692,0.000136,1.21e-05,8.81e-06,7.62e-05,8.66e-05,4.76e-05
1980-01-03,0.00444,0.00517,0.00477,0.00105,0.000325,0.00327,6.59e-05,0.00114,4.13e-05,0.000662,...,0.00146,0.00135,0.00152,0.00219,0.000361,0.000158,6.35e-05,7.85e-05,0.000139,0.000268
1980-01-04,0.000467,0.000369,0.00034,0.000198,0.000141,0.000175,7.88e-05,0.000211,7.39e-05,0.00014,...,0.00192,0.00255,0.00239,0.00205,0.000477,0.000411,0.000214,0.0002,0.000142,0.000136
1980-01-05,0.017,0.0176,0.0174,0.0148,0.0123,0.017,0.0119,0.0147,0.0118,0.0134,...,0.0111,0.0115,0.0106,0.00959,0.00608,0.00462,0.0035,0.0034,0.00359,0.00477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,0.000664,0.000655,0.000654,0.000476,0.0004,0.000745,0.000349,0.000438,0.000339,0.000396,...,0.000338,0.00031,0.000311,0.000302,0.000281,0.000455,0.000474,0.000187,0.000196,0.000232
2018-12-28,0.000418,0.00042,0.000401,0.000253,0.000141,0.000312,0.000116,0.00026,0.000116,0.000195,...,0.000216,0.00024,0.000215,0.00024,4.55e-05,2.93e-05,5.02e-05,7.67e-05,5.77e-05,2.64e-05
2018-12-29,0.0132,0.0178,0.017,0.00817,0.00262,0.0149,0.00195,0.00748,0.0019,0.00466,...,0.000144,0.000139,0.000192,0.000225,3.25e-06,5.74e-06,5.49e-06,0,1.36e-07,1.76e-06


In [39]:
# Save the DataFrames to text files with time and separate columns for each ID
temperature_daily_max_df.to_csv(os.path.join(output_folder_path, 'TMAXobs.txt'), sep='\t', na_rep='', index_label='time')
temperature_daily_min_df.to_csv(os.path.join(output_folder_path, 'TMINobs.txt'), sep='\t', na_rep='', index_label='time')